In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/home/kazuki_onodera/Python')
import lgbmextension as ex
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import gc
import EDA
import utils

In [2]:
SEED = 71

In [3]:
X = pd.concat([utils.read_pickles('../data/101_train'), 
               utils.read_pickles('../data/102_train'), 
               utils.read_pickles('../data/103_train'), 
               utils.read_pickles('../data/104_train'), 
               utils.read_pickles('../data/105_train')], axis=1)
y = utils.read_pickles('../data/label').TARGET
X.shape

100%|██████████| 20/20 [00:00<00:00, 41.57it/s]


(307511, 2625)

In [4]:
param = {
         'objective': 'binary',
         'metric': 'auc',
         'learning_rate': 0.05,
         'max_depth': -1,
         'num_leaves': 127,
         'max_bin': 100,
         'colsample_bytree': 0.5,
         'subsample': 0.5,
         'nthread': 64,
         'bagging_freq': 1,
         
         'seed': SEED
         }


categorical_feature = ['NAME_CONTRACT_TYPE',
                     'CODE_GENDER',
                     'FLAG_OWN_CAR',
                     'FLAG_OWN_REALTY',
                     'NAME_TYPE_SUITE',
                     'NAME_INCOME_TYPE',
                     'NAME_EDUCATION_TYPE',
                     'NAME_FAMILY_STATUS',
                     'NAME_HOUSING_TYPE',
                     'OCCUPATION_TYPE',
                     'WEEKDAY_APPR_PROCESS_START',
                     'ORGANIZATION_TYPE',
                     'FONDKAPREMONT_MODE',
                     'HOUSETYPE_MODE',
                     'WALLSMATERIAL_MODE',
                     'EMERGENCYSTATE_MODE']


# CV ( benchmark )

In [5]:
dtrain = lgb.Dataset(X, y, categorical_feature=categorical_feature)


In [6]:
ret = lgb.cv(param, dtrain, 9999, nfold=5,
             early_stopping_rounds=50, verbose_eval=10,
             seed=SEED)


[10]	cv_agg's auc: 0.740779 + 0.0016119
[20]	cv_agg's auc: 0.746883 + 0.00202812
[30]	cv_agg's auc: 0.751679 + 0.00221784
[40]	cv_agg's auc: 0.756304 + 0.00200631
[50]	cv_agg's auc: 0.75964 + 0.00210692
[60]	cv_agg's auc: 0.762509 + 0.00232942
[70]	cv_agg's auc: 0.764883 + 0.00244744
[80]	cv_agg's auc: 0.766723 + 0.00265349
[90]	cv_agg's auc: 0.768107 + 0.00289381
[100]	cv_agg's auc: 0.769507 + 0.00249896
[110]	cv_agg's auc: 0.77081 + 0.00258605
[120]	cv_agg's auc: 0.771975 + 0.00263665
[130]	cv_agg's auc: 0.77297 + 0.0025366
[140]	cv_agg's auc: 0.773796 + 0.00253775
[150]	cv_agg's auc: 0.77417 + 0.00255804
[160]	cv_agg's auc: 0.774633 + 0.00269693
[170]	cv_agg's auc: 0.774921 + 0.00269006
[180]	cv_agg's auc: 0.775134 + 0.00260927
[190]	cv_agg's auc: 0.775415 + 0.00267445
[200]	cv_agg's auc: 0.775457 + 0.00275487
[210]	cv_agg's auc: 0.775645 + 0.00271504
[220]	cv_agg's auc: 0.77562 + 0.00292146
[230]	cv_agg's auc: 0.775855 + 0.00282503
[240]	cv_agg's auc: 0.775864 + 0.00283696
[250]	cv

# CV ( fillna() & inf )

In [7]:
tmp = X.isnull().sum()
col = tmp[tmp>0].index
for c in col:
    X[c] = X[c].fillna(X[c].min()-1)

In [8]:
tmp = X.isnull().sum()
tmp.sum()

0

In [30]:
tmp = X.max()
col = tmp[tmp==np.inf].index
for c in col:
    val = X[X[c]!=np.inf][c].max()+1
    X[c] = X[c].replace(np.inf, val)

In [9]:
X.columns.duplicated().sum()

0

In [10]:
dtrain = lgb.Dataset(X, y, categorical_feature=categorical_feature)
gc.collect()

0

In [11]:
ret2 = lgb.cv(param, dtrain, 9999, nfold=5,
             early_stopping_rounds=50, verbose_eval=10,
             seed=SEED)

[10]	cv_agg's auc: 0.741981 + 0.00289933
[20]	cv_agg's auc: 0.74803 + 0.00214011
[30]	cv_agg's auc: 0.752263 + 0.00184612
[40]	cv_agg's auc: 0.757867 + 0.00242273
[50]	cv_agg's auc: 0.760872 + 0.00302879
[60]	cv_agg's auc: 0.763172 + 0.00264268
[70]	cv_agg's auc: 0.76528 + 0.0029898
[80]	cv_agg's auc: 0.767386 + 0.00287553
[90]	cv_agg's auc: 0.768754 + 0.00307957
[100]	cv_agg's auc: 0.769756 + 0.00294877
[110]	cv_agg's auc: 0.770717 + 0.0027935
[120]	cv_agg's auc: 0.771514 + 0.00256849
[130]	cv_agg's auc: 0.772207 + 0.00259748
[140]	cv_agg's auc: 0.773274 + 0.0024175
[150]	cv_agg's auc: 0.773882 + 0.00242034
[160]	cv_agg's auc: 0.774134 + 0.00249986
[170]	cv_agg's auc: 0.774595 + 0.0025373
[180]	cv_agg's auc: 0.774727 + 0.00249258
[190]	cv_agg's auc: 0.774923 + 0.00254973
[200]	cv_agg's auc: 0.775222 + 0.00266014
[210]	cv_agg's auc: 0.775266 + 0.0027051
[220]	cv_agg's auc: 0.775195 + 0.00271847
[230]	cv_agg's auc: 0.77545 + 0.00278377
[240]	cv_agg's auc: 0.775572 + 0.0027731
[250]	cv_a

# CV ( PCA )

In [31]:
pca = PCA(n_components=1500)
dtrain = lgb.Dataset(pca.fit_transform(X), y, 
                     categorical_feature=categorical_feature)
gc.collect()

1000

In [36]:
pca = PCA(n_components=1500)

In [38]:
X_cat = X[categorical_feature]
X_num = pca.fit_transform(X.drop(categorical_feature, axis=1))

In [47]:
gc.collect()

246

In [43]:
pd.DataFrame(X_num).head()

,0,1,2,3,4,5,6,7,8,9,...,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499
0,-1.852692e+06,-3.068010e+06,9.386097e+05,5.818899e+04,1.733445e+06,3.168378e+05,6.519920e+05,179439.037624,257033.685546,17193.081365,...,0.044971,-0.040323,-0.032767,-0.033414,-0.229243,-0.007199,-0.096890,0.051000,0.002443,0.312487
1,-8.549972e+05,-2.214985e+06,1.360000e+06,1.027540e+06,-3.136968e+06,-1.830957e+06,4.276632e+05,-872059.105195,865020.289589,579666.958081,...,0.134151,0.229581,-0.108799,-0.011706,0.116983,0.122535,-0.285338,-0.033055,0.054429,-0.016583
2,-4.105990e+06,-3.480454e+06,1.025091e+06,5.706122e+05,2.485979e+06,-1.363739e+05,2.373984e+05,170905.224228,-3670.388686,-123777.989023,...,-0.064362,0.174151,0.067130,0.095944,0.121032,0.065967,-0.077087,0.003527,0.047974,-0.195310
3,-1.241795e+07,1.446734e+07,-3.688725e+06,-1.912370e+06,-4.500337e+06,-2.697221e+05,-1.022706e+06,-73895.101934,44319.251285,-341696.446474,...,0.062503,-0.107708,0.160168,0.215140,-0.055967,0.221178,-0.268846,0.139714,-0.325760,-0.313922
4,-4.002438e+06,-3.501565e+06,1.014233e+06,6.190330e+05,-7.612544e+04,-7.412476e+05,1.494064e+05,89785.005627,56501.901096,-158270.266849,...,-0.162115,0.081174,-0.495526,-0.262172,-0.061510,0.129055,0.273409,0.169429,-0.058866,-0.051540


In [44]:
dtrain = lgb.Dataset(pd.concat([X_cat, pd.DataFrame(X_num)], axis=1), y, 
                     categorical_feature=categorical_feature)
gc.collect()

282

In [46]:
ret = lgb.cv(param, dtrain, 9999, nfold=5,
             early_stopping_rounds=50, verbose_eval=10,
             seed=SEED)

[10]	cv_agg's auc: 0.668081 + 0.00301386
[20]	cv_agg's auc: 0.679545 + 0.00338714
[30]	cv_agg's auc: 0.686735 + 0.0027056
[40]	cv_agg's auc: 0.692475 + 0.00254803


KeyboardInterrupt: 

# CV ( std scalar & PCA )

In [48]:
sc = StandardScaler()
pca = PCA(n_components=1500)

In [49]:
X_cat = X[categorical_feature]
X_num = pca.fit_transform(sc.fit_transform(X.drop(categorical_feature, axis=1)))

In [50]:
dtrain = lgb.Dataset(pd.concat([X_cat, pd.DataFrame(X_num)], axis=1), y, 
                     categorical_feature=categorical_feature)
gc.collect()

7

In [51]:
ret = lgb.cv(param, dtrain, 9999, nfold=5,
             early_stopping_rounds=50, verbose_eval=10,
             seed=SEED)

[10]	cv_agg's auc: 0.67718 + 0.00113632
[20]	cv_agg's auc: 0.686624 + 0.00168406
[30]	cv_agg's auc: 0.693288 + 0.000892071
[40]	cv_agg's auc: 0.697641 + 0.00115443
[50]	cv_agg's auc: 0.702805 + 0.00142697
[60]	cv_agg's auc: 0.707738 + 0.00220384
[70]	cv_agg's auc: 0.711586 + 0.0021631
[80]	cv_agg's auc: 0.714801 + 0.0024262
[90]	cv_agg's auc: 0.717651 + 0.00221548
[100]	cv_agg's auc: 0.719779 + 0.00179951
[110]	cv_agg's auc: 0.722205 + 0.00142931
[120]	cv_agg's auc: 0.723679 + 0.00180647
[130]	cv_agg's auc: 0.7257 + 0.00183623
[140]	cv_agg's auc: 0.726584 + 0.0017682
[150]	cv_agg's auc: 0.727525 + 0.00190254
[160]	cv_agg's auc: 0.728643 + 0.00183327
[170]	cv_agg's auc: 0.729575 + 0.0019946
[180]	cv_agg's auc: 0.730297 + 0.00169207
[190]	cv_agg's auc: 0.730984 + 0.00189294
[200]	cv_agg's auc: 0.731643 + 0.00187303
[210]	cv_agg's auc: 0.732184 + 0.00214037
[220]	cv_agg's auc: 0.732674 + 0.00219697
[230]	cv_agg's auc: 0.733064 + 0.00224419
[240]	cv_agg's auc: 0.733226 + 0.00218655
[250]	c

In [52]:
gc.collect()

43